<a href="https://colab.research.google.com/github/darkshadow013/Land-cover-ML/blob/master/ip_hyperparameter_tuning_p_s_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
"""!git clone -l -s git://github.com/darkshadow013/Land-cover-ML akshay-poria
%cd akshay-poria
!ls"""

In [ ]:
import scipy.io as si         # for inputing matlab files
import numpy as np            # Linear Algebra tools
from random import shuffle    # for shuffling dataset
import pandas as pd           # for csv files and dataframes
import matplotlib.pyplot as plt

In [ ]:
"""mat_x = si.loadmat('/content/akshay-poria/Data/Indian_pines.mat')['indian_pines']        # shape 145*145*220
mat_y = si.loadmat('/content/akshay-poria/Data/Indian_pines_gt.mat')['indian_pines_gt']  # shape 145*145
print(mat_x.shape,mat_y.shape)"""

In [ ]:
mat_x = si.loadmat('E:/GITHUB/Akshay poriya/Data/Indian_pines_corrected.mat')['indian_pines_corrected']        # shape 145*145*220
mat_y = si.loadmat('E:/GITHUB/Akshay poriya/Data/Indian_pines_gt.mat')['indian_pines_gt']  # shape 145*145
print(mat_x.shape,mat_y.shape)

In [ ]:
h, w, b = 145, 145, 200           # height,width and band of image
P_S = 7                           # patch size (window size)
oversampling = True               # Turn true to make distribution of samples across various classes to be uniform
no_of_patch = 250                # if oversampling is true enter no of patches for each class

In [ ]:
x_np, y_np = np.array(mat_x), np.array(mat_y)
x, y = x_np.reshape(h,w,b), y_np.reshape(h,w)       # x and y are equal to x_np and y_np

In [ ]:
x = x.astype(float)
for i in range(b):
  x[:,:,i] = x[:,:,i]/np.max(x[:,:,i])

In [ ]:
pad_width = int( (P_S-1)/2 )
padded_x = np.pad(x,[(pad_width,pad_width),(pad_width,pad_width),(0,0)],'edge')
print(padded_x.shape)

In [ ]:
"""from sklearn.decomposition import PCA 
  
# Let's say, components = 2 
pca = PCA(n_components = 50) 
pca.fit(padded_x) 
x_pca = pca.transform(padded_x) 
  
x_pca.shape """

In [ ]:
X,Y = [],[]
for i in range(h):
  for j in range(w):
    if(y[i][j]!= 0):
      patch = padded_x[i:i+P_S,j:j+P_S,:]
      X.append(patch)
      Y.append(y[i][j]-1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.7,stratify=Y)

In [ ]:
u_c = np.unique(y_train,return_counts=True)
print(u_c)
plt.bar( u_c[0],u_c[1], align='center', alpha=0.5)
plt.title('Distribution of samples before resampling')
plt.show()

In [ ]:
output_classes = len(np.unique(y_np))-1
classes = []
if(oversampling):
  
  for i in range(output_classes):
    classes.append([])
  for i in range(len(y_train)):
    classes[y_train[i]].append(x_train[i])
  for c in range(output_classes):
    temp = classes[c]
    print(len(classes[c]))
    for i in range( int( no_of_patch / len(classes[c]) ) ):
      classes[c]+=temp
    shuffle(classes[c])
    classes[c] = classes[c][0:no_of_patch]
  x_train,y_train = [],[]
  for c in range(output_classes):
    x_train.extend( classes[c] )
    for i in range(len(classes[c])):
      y_train.append(c)
  randomize = np.arange(len(x_train))
  np.random.shuffle(randomize)
  x_train = [x_train[i] for i in randomize]
  y_train = [y_train[i] for i in randomize]

In [ ]:
u_c = np.unique(y_train,return_counts=True)
print(u_c)
plt.bar( u_c[0],u_c[1], align='center', alpha=0.5)
plt.title('Distribution of samples after resampling')
plt.show()

In [ ]:
x_train,x_test,y_train,y_test = np.array(x_train),np.array(x_test),np.array(y_train),np.array(y_test)

In [ ]:
print(x_train.shape)

In [ ]:
patch1 = np.zeros((x_train.shape[0],P_S*P_S*b))
patch2 = np.zeros((x_test.shape[0],P_S*P_S*b))

In [ ]:
print(patch1,patch1.shape)

In [ ]:
for i in range(patch1.shape[0]):
  patch1[i,:] = x_train[i,:,:,:].flatten('C')
for i in range(patch2.shape[0]):
  patch2[i,:] = x_test[i,:,:,:].flatten('C')

x_train = patch1
x_test = patch2

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import svm
import time

**Hyperparameter Tunning**

In [ ]:
sigma = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
C = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]

acc = list()
sigma_c = list()
max_acc=0.0
for each in sigma:
    for each_c in C:
      start = time.time()
      clf = svm.SVC(kernel='rbf', gamma = 1/(2*(each**2)), C=each_c, decision_function_shape='ovr')
      clf.fit(x_train, y_train)
      accuracy = clf.score(x_test, y_test)
      acc.append(accuracy)
      sigma_c.append((each, each_c))
      print('Sigma: %f, C: %f, accuracy: %f, Time: %f sec' % (each, each_c, accuracy, time.time()-start))

In [ ]:
index = np.argmax(acc)

sigma_max, c_max = sigma_c[index]

print('The optimal value of sigma is: {}'.format(sigma_max))
print('The optimal value of C is: {}'.format(c_max))
print('The optimal value of accuracy is: {}'.format(acc[index]))

**Best Model**

In [ ]:
sigma = 1
gamma = 1/(2 * sigma**2)

In [ ]:
clf_best = svm.SVC(kernel='rbf', gamma=gamma, C=30, decision_function_shape='ovr')

In [ ]:
clf_best.fit(x_train, y_train)

In [ ]:
y_pred = clf_best.predict(x_test)

In [ ]:
results = confusion_matrix(y_test,y_pred)
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(y_test, y_pred)) 
print ('Report : ')
print (classification_report(y_test, y_pred))

**Classified Image**

In [ ]:
X1 = np.array(X)
X2 = np.zeros((X1.shape[0],P_S*P_S*b))
for i in range(X2.shape[0]):
  X2[i,:] = X1[i,:,:,:].flatten('C')

In [ ]:
# Function to extract patche at h_index,w_index 

def patch_at_index(h_index,w_index):
  patch = padded_x[h_index:h_index+P_S,w_index:w_index+P_S,:]
  patch = patch.flatten('C')
  patch = patch.reshape(1,patch.shape[0])
  return patch

In [ ]:
y_hat = np.zeros((h,w),dtype=int)

for i in range(h):
  for j in range(w):
    if(i%2==0 and j==0):
      print(i)
    patch = patch_at_index(i,j)
    y_hat[i][j] =  clf_best.predict(patch) +1

In [ ]:
pip install spectral

In [ ]:
import spectral
spectral.imshow(classes = y,figsize=(10,10))
spectral.imshow(classes = y_hat,figsize=(10,10))

In [ ]:
plt.plot(acc, label='Training Accuracy')

In [ ]:
from mpl_toolkits import mplot3d

ax = plt.axes(projection="3d")
zzz = np.array(sigma_c)
ax.plot3D(zzz[:,0],zzz[:,1], acc)